# Homography

## Librairies

In [1]:
from PIL import Image
import numpy as np
from time import time

## Images et matrices

In [2]:
simp = Image.open('simpsons.jpeg')
bus = Image.open('bus.jpeg')
simp_mat = np.array(simp)
bus_mat = np.array(bus)

## Fonction basée sur Tkinter pour sélectionner les coins de l'affiche

In [3]:
import tkinter as tk
from PIL import Image, ImageTk

class PointSelector:
    def __init__(self, image_path):
        self.image = Image.open(image_path)
        self.root = tk.Tk()
        self.root.title("Point Selector")
        self.canvas = tk.Canvas(self.root, width=self.image.width, height=self.image.height)
        self.canvas.pack()
        self.photo = ImageTk.PhotoImage(self.image)
        self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)
        self.points = []
        self.canvas.bind("<Button-1>", self.on_click)
        
        self.quit_button = tk.Button(self.root, text="Quitter", command=self.root.quit)
        self.quit_button.pack()
        
        self.root.mainloop()

    def on_click(self, event):
        x, y = event.x, event.y
        self.points.append((x, y))
        self.canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="red")
        print(f"Point sélectionné: x : {y}, y : {x}") ## x et y sont inversés dans numpy

In [4]:
PointSelector('bus.jpeg')

Point sélectionné: x : 210, y : 584
Point sélectionné: x : 232, y : 798
Point sélectionné: x : 469, y : 808
Point sélectionné: x : 520, y : 595


## Résolution du système d'équation par la méthode des moindres carrés

In [5]:
### Méthode des moindres carrés
x1, y1 = 0,0
x2, y2 = 0, simp_mat.shape[1]
x3, y3 = simp_mat.shape[0], simp_mat.shape[1]
x4, y4 = simp_mat.shape[0], 0
coo = [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]

xt1, yt1 = 210, 583
xt2, yt2 = 231, 799
xt3, yt3 = 469, 809
xt4, yt4 = 520, 597
coot = [[xt1, yt1], [xt2, yt2], [xt3, yt3], [xt4, yt4]]


T = []
for i in range(len(coo)):
    T.append([coo[i][0], coo[i][1], 1, 0, 0, 0, -coo[i][0]*coot[i][0], -coo[i][1]*coot[i][0]])
    T.append([0, 0, 0, coo[i][0], coo[i][1], 1, -coo[i][0]*coot[i][1], -coo[i][1]*coot[i][1]])
T = np.array(T)

In [6]:
B = np.array(np.meshgrid(np.array(coot)))
H = np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(T), T)), np.transpose(T)), np.transpose(B))
H = np.append(H,1)
H = H.reshape((3,3))
print(H)

[[2.08244712e-01 9.11076796e-02 2.10000000e+02]
 [1.11450509e-02 4.58493662e-01 5.83000000e+02]
 [3.03470287e-06 3.03496449e-04 1.00000000e+00]]


## Question 2 : Homography de simpson vers bus

### Fonction d'homography

In [7]:
def homography(source, destination, matrice):
    for i in range(source.shape[0]):
        for j in range(source.shape[1]):
            coo = np.dot(matrice, np.array([i,j,1]))
            x,y = int(coo[0]/coo[2]), int(coo[1]/coo[2])
            destination[x, y] = source[i, j]
                
    destination = destination/np.max(destination)*255
    return destination

### Formation de l'image transformée

In [10]:
def q2():
    bus_mat = np.array(bus)
    bus_mat_trans = homography(simp_mat, bus_mat, H)

    bus_transf = Image.fromarray(np.uint8(bus_mat_trans))
    return bus_transf


In [11]:

bus_transf = q2()
bus_transf.show()

## Question 3 : homographie de l'affiche du bus vers les simpsons

In [13]:

xt1, yt1 = 210, 583
xt2, yt2 = 231, 799
xt3, yt3 = 520, 597
xt4, yt4 = 469, 809
coot = [[xt1, yt1], [xt3, yt3],[xt4, yt4], [xt2, yt2]]
bus_mat = np.array(bus)

# Fonction pour vérifier si un point est à l'intérieur d'un polygone
def point_inside_polygon(x, y, poly):
    n = len(poly)
    
    inside = False
    p1x, p1y = poly[0]
    for i in range(n + 1):
        p2x, p2y = poly[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y
    return inside

    
def q3() : 
    M = np.ones_like(bus_mat)
    # Parcourir tous les pixels dans le rectangle englobant du polygone
    for i in range(min(xt1, xt2, xt3, xt4), max(xt1, xt2, xt3, xt4)):
        for j in range(min(yt1, yt2, yt3, yt4), max(yt1, yt2, yt3, yt4)):
            if point_inside_polygon(i, j, coot):
                coo = np.dot(np.linalg.inv(H), np.array([i,j,1]))
                x,y = int(coo[0]/coo[2]), int(coo[1]/coo[2])
                bus_mat[i,j] = simp_mat[x,y]

    bus_img = Image.fromarray(np.uint8(bus_mat))
    return bus_img


In [14]:
bus_img = q3()
bus_img.show()

## Portion de code pour calculer le temps moyen mis avec l'utilisation de chacune des deux méthodes

In [15]:
def temps(f):
    T=0
    for k in range(100):
        t1 = time()
        f()
        T+= time() - t1
    return(T/100)


In [16]:
print(temps(q2))
print(temps(q3))